In [26]:
from transformers import ViTImageProcessor, ViTForImageClassification

from unsplash.api import Api
from unsplash.auth import Auth
import openai

import requests
import os
from PIL import Image

checklist = []
lstofid = []
images = []


In [27]:
def ai_check(mainidea, simmilarity):     
    """
    Take the main idea and the simmilarity and return the answer from openai.
    @param mainidea - the main idea of the presentation           
    @param simmilarity - the simmilarity of the presentation           
    @return The answer from openai.           
    """
    
    return (openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"i am creating a presentation to present about {mainidea}, i created some photos about stuff that should relate in any way to the subject. tell me if this subject: '{simmilarity}' has anything to do with my main idea about my presentation. for example: ai is associated with computers, computers are associated with laptops, laptops are associated with learning and etc. another example: cats are associated with pets, pets is associated with dogs, and etc. another example is: if i asked for israeli defense army and computers, if you get flag as a subject or army or war it will be a yes because it is associated with all of that. another example is: spoons is associated with food, dumplings is associated with korean food which is associated with korea.another example is that forest is associated with monkeys, which are associated with banans, which is associated with trees. another example: nike is associated with clothing, shoes, socks, etc, shoes are associated wit socks, and socks are associated with nike at the end. another example, you might not understand, but for example, inflation has several meanings, one of each is high gas prices or money on the ground, there are more examples that have the vibe of this stuff which is all related to inflation, this does not only about inflation, there are other subjects that are also related to that and also have several meanings, respect that. all of those are simmilar things that sort of relate to eachother in some way. i gave you inputs based on that idea. You get the idea; return yes/no. ",
                           temperature=0.3,
                           max_tokens=100,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]).split("|")


In [28]:
def get_image_ai(mainidea, amount):
    """
    Download an image from the openai API.
    @param mainidea - the main idea of the presentation           
    @param amount - the amount of images to download           
    """
    for number in range(int(amount)):
        response = openai.Image.create(
          prompt=f"a basic photo, make it look realistic, suitable for a presentation, related to the subject: {mainidea}",
          n=4,
          size="1024x1024"
        )
        image_url = response['data'][0]['url']
        def download_image(url, headers):
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                with open(f"image_presentation_{number}.png", "wb") as file:
                    file.write(response.content)
            else:
                print(response.status_code)
        download_image(image_url, headers = {
                "User-Agent": "Chrome/51.0.2704.103",
            }
        )


In [29]:
def image_download(url, number):
    """
    Download an image from the url and save it to the file system.
    @param url - the url of the image to download.
    @param number - the number of the image to save.
    """
    response = requests.get(url)
    with open(f"image_presentation_{number}.png", "wb") as file:
        file.write(response.content)


In [30]:
def check_similar(url):
    """
    Take the url of an image and check if it is similar to any of the images in the dictionary.
    @param url - the url of the image to check against the dictionary
    @returns the predicted class of the image
    """
    image = Image.open(requests.get(url, stream=True).raw)
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
    model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    checklist.append(model.config.id2label[predicted_class_idx])


In [31]:

def get_image_unsplash(mainidea, amount, width = 0, height = 0):
    """
    Given a main idea, amount of images, and optional width and height, download the images from Unsplash.com.
    @param mainidea - the main idea for the image search.
    @param amount - the amount of images to download.
    @param width - the width of the image.
    @param height - the height of the image.
    """
    for photo in (api.search.photos(mainidea)['results'])[0:amount]:
        image = api.photo.get((str(photo)).split("=")[1].replace("'", "").replace(")", ""), width = width, height = height)
        lstofid.append((str(image)).split("=")[1].replace("'", "").replace(")", ""))
    for number in range(int(amount)):
        url = f"""https://unsplash.com/photos/{lstofid[number]}/download"""
        check_similar(url) # f"unsplash_{number}")
        image_download(url, number)
        print()

In [32]:
answers = {}
    #Remove any old images from the directory.
number = 0

while number == number:
    try:
        os.remove(f"image_presentation_{number}.png")
        number += 1
    except:
        break

##################################
mainidea = "Cats"
num_of_images = 3
#################################

get_image_unsplash(mainidea.title(), num_of_images + 2)
    #Get the images from unsplash.com and check them against the AI.
    #@param mainidea - the main idea of the image
    #@param num_of_images - the number of images to get
    #@returns the images that are relevant to the AI
for place in range(len(checklist)):
    answers.update({place : ai_check(mainidea, checklist[place])})
    
for key, value in answers.items():
    print(key)
    if 'yes' in str(" ".join(value).lower()):
        images.append(key)
        if len(images) >= num_of_images:
            print("Succesfull")
            break
if len(images) < num_of_images:
    print("AI")
    get_image_ai(mainidea.title(), num_of_images)
        







0
1
2
Succesfull


In [33]:
checklist

['Persian cat',
 'Persian cat',
 'Persian cat',
 'tabby, tabby cat',
 'Egyptian cat']